In [83]:
import os
import sys
import time
import math
import csv

import multiprocessing
import logging
from itertools import product, repeat
import optparse

import rasterio
from rasterio import windows
from rasterio.enums import Resampling
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
import numpy as np
import fiona
from shapely import geometry
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def grid_calc(width, height, stride):
    #get all the upper left (ul) pixel values of our chips. Combine those into a list of all the chip's ul pixels. 
    x_chips = width // stride
    x_ul = [stride * s for s in range(x_chips)]

    y_chips = height // stride
    y_ul = [stride * s for s in range(y_chips)]

    #xy_ul = product(x_ul, y_ul) #this is the final list of ul pixel values
    #print(f"xy_ul: {len(xy_ul)}")

    xy_ul = []
    for x in x_ul:
        for y in y_ul:
            xy_ul.append((x, y))

    #print(f"x/y ul len: {len(x_ul)} / {len(y_ul)}")
    #print(f"y * x = {len(x_ul) * len(y_ul)}")
    #print(f"xy_ul len: {len(xy_ul)}")

    return xy_ul

In [3]:
def build_window(in_src, upper_left_pix_coords, in_chip_size, in_chip_stride):
    row_start = upper_left_pix_coords[0]
    row_stop = upper_left_pix_coords[0] + in_chip_size
    col_start = upper_left_pix_coords[1]
    col_stop = upper_left_pix_coords[1] + in_chip_size
    
    #print(row_start, col_start, row_stop, col_stop)
    
    out_window = windows.Window.from_slices((row_start, row_stop), (col_start, col_stop))
    
    '''
    out_window = windows.Window(col_off = upper_left_pix_coords[0],
                                row_off = upper_left_pix_coords[1],
                                width=in_chip_size,
                                height=in_chip_size)'''
    
    #out_win_transform = in_src.window_transform(out_window)
    out_win_transform = windows.transform(out_window, in_src.transform)
    #print(out_win_transform)
    
    
    row_id = upper_left_pix_coords[0] // in_chip_stride
    col_id = upper_left_pix_coords[1] // in_chip_stride
    out_win_id = f'{row_id}_{col_id}'
    
    out_win_bounds = windows.bounds(out_window, out_win_transform)
    #print(out_win_bounds)
    
    return out_window, out_win_transform, out_win_bounds, out_win_id
    

In [4]:
def make_gdf(polygons, attr_dict, out_crs, out_gdf_path='none'):
    #print(f"out_gdf_path: {out_gdf_path}")
    gs = gpd.GeoSeries(polygons)
    
    df = pd.DataFrame(data=attr_dict)
    
    gdf = gpd.GeoDataFrame(df, geometry=gs)
    gdf.crs=out_crs
    
    #optionally write a file if the path was provided. NOTE: COULD EXPAND THIS TO HANDLE FORMATS OTHER THAN GPKG.
    if out_gdf_path != 'none':
        if os.path.exists(os.path.dirname(os.path.abspath(out_gdf_path))):
            print(f"Writing: {out_gdf_path}")
            
            gdf.to_file(out_gdf_path, driver='GPKG')
    
    #regardless of writing output, return GDF
    return gdf


In [5]:
def write_jpeg(in_data, in_count, in_size, in_win_transform, in_src_crs, in_out_path):
    #building a custom jpeg profile for our chip due to some gdal/rasterio bugs in walking from input geotiff to output jpeg
    profile={'driver': 'JPEG',
        'count': in_count,
        'dtype': rasterio.ubyte,
        'height': in_size,
        'width': in_size,
        'transform': in_win_transform,
        'crs': in_src_crs}
        
    #write the chip
    with rasterio.open(in_out_path, 'w', **profile) as dst:
        dst.write(in_data)

In [6]:
def is_nodata(in_data):
    #check to make sure our chip isn't made up of all zeros (nodata)
    zeros = np.zeros_like(in_data)
    #check if they're equal, return result
    if np.array_equal(in_data, zeros):
        return True
    else:
        return False

In [7]:
def coords_2_pix(in_bounds, in_affine):
    xmin = in_bounds[0]
    ymin = in_bounds[1]
    xmax = in_bounds[2]
    ymax = in_bounds[3]
    
    xs = (xmin, xmax)
    ys = (ymin, ymax)
    
    pix_coords = rasterio.transform.rowcol(in_affine, xs, ys)
    
    pix_bounds = (pix_coords[0][1], pix_coords[1][1], pix_coords[0][0], pix_coords[1][0])
    
    return pix_bounds

def coords_2_pix_gdf(gdf):
    gdf['x_min'] = gdf.bounds.minx
    gdf['y_min'] = gdf.bounds.miny
    gdf['x_max'] = gdf.bounds.maxx
    gdf['y_max'] = gdf.bounds.maxy
    
    gdf['px_x_min'] = (gdf['x_min'] - gdf['a2']) // gdf['a0']
    gdf['px_x_max'] = (gdf['x_max'] - gdf['a2']) // gdf['a0']
    
    #NOTE: in coordinates we the origin as the bottom left corner. In pixel coordinates, we set the origin at the top left corner.
    #Anyways, we're flipping our pix min/max here so that our origin starts at the top right corner.
    gdf['px_y_max'] = (gdf['y_min'] - gdf['a5']) // gdf['a4']
    gdf['px_y_min'] = (gdf['y_max'] - gdf['a5']) // gdf['a4']
    
    return gdf

def pix_2_coords(in_bounds, in_affine):
    xmin = in_bounds[0]
    ymin = in_bounds[1]
    xmax = in_bounds[2]
    ymax = in_bounds[3]
    
    xs = (xmin, xmax)
    ys = (ymin, ymax)
    
    pix_coords = rasterio.transform.xy(in_affine, xs, ys)
    
    pix_bounds = (pix_coords[0][1], pix_coords[1][1], pix_coords[0][0], pix_coords[1][0])
    return pix_bounds

In [8]:
def return_intersection(in_tindex, in_annotations, unique_annotation_id):
    inter = gpd.overlay(in_tindex, in_annotations)
    inter['intersect_area'] = inter['geometry'].area
    filter_partial_annotations = inter[inter['intersect_area'] == 1.0]
    remove_duplicates = filter_partial_annotations.drop_duplicates(subset=unique_annotation_id)
    
    return remove_duplicates

In [56]:
def create_cindex(in_file, in_size, in_stride, in_out_dir):
    basename = os.path.splitext(os.path.basename(in_file))[0]
    print(basename)
    #print(f"{in_file}, {in_size}, {in_stride}, {in_out_dir}")
    gdfs = []
    pos_files = []
    neg_files = []
    with rasterio.open(in_file, 'r') as src:

        print(f"src height/width: {src.height}/{src.width}")
        print(f"src bounds: {src.bounds}")
        print(f"src transform: {src.transform}")
        
        upper_left_grid = grid_calc(src.width, src.height, in_stride)
        
        for ul in upper_left_grid:
            #note, we're currently working with slices because I can't make col_off, row_off work. Code needs to be reworked to naturally work with slices
            col_start = ul[0]
            col_stop = ul[0] + in_size
            row_start = ul[1]
            row_stop = ul[1] + in_size
            slices = (col_start, row_start, col_stop, row_stop)
                
            win, win_transform, win_bounds, win_id = build_window(src, ul, in_size, in_stride)

            #NOTE: I had to write my own affine lookup to get bounding boxs from windows (pix_2_coords). Rasterio's windows.bounds(win, win_transform) 
            #caused every overlpping tile to shift 256 pix in the x and y direction (removed overlap, doubled area covered by chip tindex)
            #therefore, the win_bounds variable above should not currently be used. I need to investigate further.
            
            ret = pix_2_coords(slices, src.transform)
            
            #create and store the chip's geometry (the bounding box of the image chip)
            envelope = geometry.box(*ret)
            geometries=[]
            geometries.append(envelope)
            
            #store the image basename. No real reason, just comes in handy alot.
            attr_basename=[]
            attr_basename.append(in_file)
            
            #store chip name as an attribute in the cindex attribute table
            chip_name = f"{basename}_{win_id}"
            attr_filename = []
            attr_filename.append(chip_name)
            
            # store affine values as attributes in the cindex attribute table
            px_width = []
            row_rot = []
            col_off = []
            col_rot = []
            px_height = []
            row_off = []
            
            px_width.append(win_transform[0])
            row_rot.append(win_transform[1])
            col_off.append(win_transform[2])
            col_rot.append(win_transform[3])
            px_height.append(win_transform[4])
            row_off.append(win_transform[5])
            
            #create a single chip feature with attributes and all
            attr_dict = {}
            attr_dict['basename'] = attr_basename
            attr_dict['filename'] = attr_filename
            attr_dict['a0'] = px_width
            attr_dict['a1'] = row_rot
            attr_dict['a2'] = col_off
            attr_dict['a3'] = col_rot
            attr_dict['a4'] = px_height
            attr_dict['a5'] = row_off
        
            chip_gdf = make_gdf(geometries, attr_dict, src.crs)

            #append our single chip feature to a list of all chips. Later we will merge all the single chips into a big chip index (cindex).
            gdfs.append(chip_gdf)
            
    #merge all those little chip features together into our master cindex for the input image 
    cindex_gdf_path = os.path.join(in_out_dir, f"{basename}_chip_tindex.gpkg")
    cindex_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    cindex_gdf.crs = src.crs
    cindex_gdf.to_file(cindex_gdf_path, driver='GPKG')

    return cindex_gdf

In [57]:
def append_coords_and_pix_to_gdf(in_gdf, in_transform):
    in_gdf['x_min_coords'] = 0
    in_gdf['y_min_coords'] = 0
    in_gdf['x_max_coords'] = 0
    in_gdf['y_max_coords'] = 0
    
    in_gdf['x_min'] = 0
    in_gdf['y_min'] = 0
    in_gdf['x_max'] = 0
    in_gdf['y_max'] = 0
    
    for i, row in pos_gdf.iterrows():
        #get the mins and the maxs
        xmin = row.geometry.bounds[0]
        xmax = row.geometry.bounds[2]
        ymin = row.geometry.bounds[1]
        ymax = row.geometry.bounds[3]
        bounds = (xmin, ymin, xmax, ymax)

        #convert our northing/easting min/maxs into pixel coordinates using the chip's affine transformation matrix
        img_coords = coords_2_pix(bounds, in_transform)

        #I think you need to create these columns in advance to make this work
        in_gdf.loc[i, 'x_min_coords'] = bounds[0]
        in_gdf.loc[i, 'y_min_coords'] = bounds[1]
        in_gdf.loc[i, 'x_max_coords'] = bounds[2]
        in_gdf.loc[i, 'y_max_coords'] = bounds[3]

        in_gdf.loc[i, 'x_min'] = bounds[0]
        in_gdf.loc[i, 'y_min'] = bounds[1]
        in_gdf.loc[i, 'x_max'] = bounds[2]
        in_gdf.loc[i, 'y_max'] = bounds[3]
        
    return in_gdf

In [58]:
def write_annotations(in_gdf, out_path='none'):
    coord_gdf = coords_2_pix_gdf(in_gdf)
    
    #set our columns/column order
    out_gdf = coord_gdf[['filename', 'x_min', 'y_min', 'x_max', 'y_max', 
                         'px_x_min', 'px_y_min', 'px_x_max', 'px_y_max', 
                         'label_name', 'label', 'label_int']]
    
    if out_path != 'none':
        out_gdf.to_csv(out_path)
    
    return out_gdf

In [99]:
def mask_raster(in_poly, src_raster, in_out_path):
    with rasterio.open(src_raster, 'r') as src:
        out_data, out_transform = mask(src, [in_poly], crop=True)
         
        write_jpeg(out_data, src.count, out_data.shape[1], out_transform, crs.crs, in_out_path)

        

In [100]:
def main(in_f, in_anno, in_chip_size, in_chip_stride, in_out_dir):
    #Chip out our image, return a cindex
    cindex = create_cindex(in_f, in_chip_size, in_chip_stride, in_out_dir)
    
    #find all the annotations that intersect each chip. Filter chips with no annotations, filter annotations that are not fully contained within a chip.
    intersect = return_intersection(cindex, in_anno, 'unique_pt_id')
    
    #generate a list of positive chips in the annotation database.
    pos_chips = intersect['filename'].unique().tolist()
    
    pos_chips_gdf = cindex[cindex['filename'].isin(pos_chips)]
    for i, row in pos_chips_gdf[['geometry', 'basename','filename']].iterrows():
        polygon = row["geometry"]
        src_raster = row['basename']
        out_raster_path = os.path.join(out_dir, f"{row['filename']}.jpg")
        
        #this also write our positive image chip to a jpeg located at out_raster_path
        mask_raster(polygon, src_raster, out_raster_path)

    #return our annotations to be bound into a island-wide annotation data set.
    return cindex, intersect, pos_chips_gdf

In [101]:
in_file_list = r"/media/ross/ssd/00_kahoolawe_dar2015/00_256x256/tif_list_1.txt"
out_dir = r"/media/ross/ssd/00_kahoolawe_dar2015/04_testing"
in_annotations = r"/media/ross/ssd/00_kahoolawe_dar2015/labels/KO_dar2015_annotations_envelopes_20191028.gpkg"

chip_size = 512
chip_stride = 256

In [66]:
with open(in_file_list, 'r') as f:
    in_paths = [line.strip() for line in f]
    
in_anno = gpd.read_file(in_annotations)

In [67]:
for f in in_paths:
    cind, inter, pos = main(f, in_anno, chip_size, chip_stride, out_dir)

kahoolawe_1639
src height/width: 14848/14848
src bounds: BoundingBox(left=755065.6217836322, bottom=2272396.285158604, right=755365.6617836322, top=2272696.325158604)
src transform: | 0.02, 0.00, 755065.62|
| 0.00,-0.02, 2272696.33|
| 0.00, 0.00, 1.00|


In [102]:
inter.head()

,basename,filename,a0,a1,a2,a3,a4,a5,unique_pt_id,island,segment,pt_id,label_int,label,label_name,geometry,intersect_area
1,/media/ross/ssd/00_kahoolawe_dar2015/00_256x25...,kahoolawe_1639_14_0,0.020207,0.0,755065.621784,0.0,-0.020207,2.272624e+06,KO-009-0134,KO,9,134,6,P,plastic,"POLYGON ((755069.956907192 2272613.815506428, ...",1.0
11,/media/ross/ssd/00_kahoolawe_dar2015/00_256x25...,kahoolawe_1639_16_0,0.020207,0.0,755065.621784,0.0,-0.020207,2.272614e+06,KO-009-0137,KO,9,137,6,P,plastic,"POLYGON ((755068.5794146496 2272608.256452742,...",1.0
4,/media/ross/ssd/00_kahoolawe_dar2015/00_256x25...,kahoolawe_1639_15_0,0.020207,0.0,755065.621784,0.0,-0.020207,2.272619e+06,KO-009-0136,KO,9,136,0,B,buoy,"POLYGON ((755065.7578133029 2272608.546474151,...",1.0
5,/media/ross/ssd/00_kahoolawe_dar2015/00_256x25...,kahoolawe_1639_15_0,0.020207,0.0,755065.621784,0.0,-0.020207,2.272619e+06,KO-009-0135,KO,9,135,0,B,buoy,"POLYGON ((755066.2661337365 2272609.440237319,...",1.0
9,/media/ross/ssd/00_kahoolawe_dar2015/00_256x25...,kahoolawe_1639_16_0,0.020207,0.0,755065.621784,0.0,-0.020207,2.272614e+06,KO-009-0141,KO,9,141,6,P,plastic,"POLYGON ((755072.4079332839 2272603.440024172,...",1.0
